#Data Collection Acrolein

In [ ]:
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
!pip install mlatom
!git clone https://gitlab.com/maxjr82/ulamdyn.git
!pip install ulamdyn/

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from typing_extensions import get_overloads
data = np.load('ws22_acrolein.npz')

for key, val in data.items():
    print(key, '->', val.shape)

coordinates_cis_trans = np.array(data['R'][:20000])
energy_cis_trans = np.array(data['E'][:20000])
grads_cis_trans = -np.array(data['F'][:20000])


atoms = []

for i in range(data['Z'].shape[0]):
    if data['Z'][i] == 6:
        atoms.append('C')
    elif data['Z'][i] == 8:
        atoms.append('O')
    elif data['Z'][i] == 1:
        atoms.append('H')

print(atoms)

R -> (120000, 8, 3)
F -> (120000, 8, 3)
Q -> (120000, 8, 1)
DP -> (120000, 3)
QP -> (120000, 3, 3)
P -> (120000, 6)
RC -> (120000, 3)
HL -> (120000, 2)
E -> (120000, 1)
Z -> (8,)
R2 -> (120000, 1)
CONF -> (120000, 1)
['C', 'O', 'C', 'C', 'H', 'H', 'H', 'H']


In [ ]:
coordinates_trans = np.array(data['R'][70000:])
energy_trans = np.array(data['E'][70000:])
grads_trans = -np.array(data['F'][70000:])

In [ ]:
coordinates_cis = np.array(data['R'][20000:70000])
energy_cis = np.array(data['E'][20000:70000])
grads_cis = -np.array(data['F'][20000:70000])

In [ ]:
coordinates_trans_train, coordinates_trans_test, en_trans_train, en_trans_test, grads_trans_train, grads_trans_test = train_test_split(coordinates_trans, energy_trans, grads_trans, train_size = 0.90, random_state=0)

In [ ]:
en_trans_train

array([[-120265.54525667],
       [-120295.6376437 ],
       [-120290.32390238],
       ...,
       [-120294.09290581],
       [-120273.4211467 ],
       [-120292.20436419]])

In [ ]:
print(en_trans_train.shape)
print(en_trans_test.shape)

(45000, 1)
(5000, 1)


In [ ]:
import ulamdyn as ulmd
import os

from mlatom.data import molecular_database, molecule
from mlatom.interfaces.torchani_interface import ani
# from mlatom.interfaces.gap_interface import gap
from mlatom import models
from mlatom.constants import Hartree2eV, Angstrom2Bohr, kcalpermol2eV
import torch

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter

import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
epots_train = en_trans_train.flatten() * kcalpermol2eV
epots_test = en_trans_test.flatten() * kcalpermol2eV
grads_train = grads_trans_train * kcalpermol2eV
grads_test = grads_trans_test * kcalpermol2eV

In [ ]:
epots_train

array([-5217.51587023, -5218.8213772 , -5218.59084958, ...,
       -5218.75436138, -5217.85755231, -5218.67243021])

In [ ]:
atoms = np.tile(np.array(1 * [6] + 1 * [8] + 2 * [6] + 4 * [1]), (45000, 1))
acrolein_train = molecular_database.from_numpy(coordinates_trans_train, atoms)

In [ ]:
acrolein_train.element_symbols.shape

(45000, 8)

In [ ]:
acrolein_train.xyz_coordinates.shape

(45000, 8, 3)

In [ ]:
dir(acrolein_train)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_uniform_cell',
 '_load',
 'add_hessian_properties',
 'add_scalar_properties',
 'add_scalar_properties_from_file',
 'add_xyz_derivative_properties',
 'add_xyz_derivative_properties_from_file',
 'add_xyz_vectorial_properties',
 'add_xyz_vectorial_properties_from_file',
 'add_xyz_vectorial_properties_from_string',
 'append',
 'atomic_numbers',
 'batches',
 'charges',
 'copy',
 'dump',
 'element_symbols',
 'filter_by_property',
 'from_numpy',
 'from_smiles_file',
 'from_smiles_string',
 'from_xyz_file',
 'from_xyz_string',
 'get_atomic_numbers',
 'get_element_symbols',

In [ ]:
acrolein_train.add_scalar_properties(epots_train, "energy")

In [ ]:
acrolein_train.add_xyz_derivative_properties(grads_train, "energy", "energy_grad")

In [ ]:
acrolein_train.size

45000

In [ ]:
acrolein_train.get_properties('energy').shape

(45000,)

In [ ]:
acrolein_train.get_xyz_derivative_properties('energy_grad').shape

(45000, 8, 3)

In [ ]:
import random
import torch
import os
import sys
import numpy as np
import pandas as pd
import ulamdyn as ulmd
import torch
import torchani
from mlatom.data import molecular_database, molecule
from mlatom.interfaces.torchani_interface import ani
from mlatom.constants import Hartree2eV, Angstrom2Bohr

In [ ]:
def set_seed(seed):
    # Set the seed for the Python random module
    random.seed(seed)

    # Set the seed for NumPy
    np.random.seed(seed)

    # Set the seed for PyTorch (CPU)
    torch.manual_seed(seed)

    # If using a GPU, set the seed for PyTorch (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.

    # Ensure deterministic behavior for some operations
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a seed value
seed = 0
set_seed(seed)

In [ ]:
hyperparams = models.hyperparameters({
        #### Training ####
        'batch_size':           models.hyperparameter(value=128, minval=1, maxval=1024, optimization_space='linear', dtype=int),
        'max_epochs':           models.hyperparameter(value=1000, minval=100, maxval=1000000, optimization_space='log', dtype=int),
        'learning_rate':                    models.hyperparameter(value=0.002, minval=0.0001, maxval=0.01, optimization_space='log'),
        'early_stopping_learning_rate':     models.hyperparameter(value=1.0E-7, minval=1.0E-6, maxval=1.0E-4, optimization_space='log'),
        'lr_reduce_patience':   models.hyperparameter(value=50, minval=16, maxval=256, optimization_space='linear'),
        'lr_reduce_factor':     models.hyperparameter(value=0.8, minval=0.1, maxval=0.9, optimization_space='linear'),
        'lr_reduce_threshold':  models.hyperparameter(value=0.0, minval=-0.01, maxval=0.01, optimization_space='linear'),
        #### Loss ####
        'force_coefficient':    models.hyperparameter(value=0.1, minval=0.05, maxval=5, optimization_space='linear'),
        'median_loss':          models.hyperparameter(value=False),
        'validation_loss_type': models.hyperparameter(value='mean_RMSE', choices=['MSE', 'mean_RMSE']),
        #### Network ####
        "neurons":              models.hyperparameter(value=[[72, 64, 64, 16]]),
        "activation_function":  models.hyperparameter(value=lambda: torch.nn.ReLU(), optimization_space='choice', choices=["CELU", "ReLU", "GELU"]),
        "fixed_layers":         models.hyperparameter(value=False),
        #### AEV ####
        'Rcr':                  models.hyperparameter(value=5.2000e+00, minval=1.0, maxval=10.0, optimization_space='linear'),
        'Rca':                  models.hyperparameter(value=3.5000e+00, minval=1.0, maxval=10.0, optimization_space='linear'),
        'EtaR':                 models.hyperparameter(value=[1.6000000e+01]),
        'ShfR':                 models.hyperparameter(value=[9.0000000e-01, 1.1687500e+00, 1.4375000e+00, 1.7062500e+00, 1.9750000e+00, 2.2437500e+00, 2.5125000e+00, 2.7812500e+00, 3.0500000e+00, 3.3187500e+00, 3.5875000e+00, 3.8562500e+00, 4.1250000e+00, 4.3937500e+00, 4.6625000e+00, 4.9312500e+00]),
        'Zeta':                 models.hyperparameter(value=[3.2000000e+01]),
        'ShfZ':                 models.hyperparameter(value=[1.9634954e-01, 5.8904862e-01, 9.8174770e-01, 1.3744468e+00, 1.7671459e+00, 2.1598449e+00, 2.5525440e+00, 2.9452431e+00]),
        'EtaA':                 models.hyperparameter(value=[8.0000000e+00]),
        'ShfA':                 models.hyperparameter(value=[9.0000000e-01, 1.5500000e+00, 2.2000000e+00, 2.8500000e+00]),
    })

In [ ]:
model_ani = ani("ani_acrolein.pt")
model_ani.hyperparameters = hyperparams

the trained ANI model will be saved in ani_acrolein.pt


In [ ]:
model_ani.train(
    molecular_database=acrolein_train,
    property_to_learn="energy",
    xyz_derivative_property_to_learn="energy_grad",
    save_model=True,
)

Sequential(
  (0): AEVComputer()
  (1): ANIModel(
    (C): Sequential(
      (0): Linear(in_features=240, out_features=72, bias=True)
      (1): ReLU()
      (2): Linear(in_features=72, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
      (6): Linear(in_features=64, out_features=16, bias=True)
      (7): ReLU()
      (8): Linear(in_features=16, out_features=1, bias=True)
    )
    (H): Sequential(
      (0): Linear(in_features=240, out_features=72, bias=True)
      (1): ReLU()
      (2): Linear(in_features=72, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
      (6): Linear(in_features=64, out_features=16, bias=True)
      (7): ReLU()
      (8): Linear(in_features=16, out_features=1, bias=True)
    )
    (O): Sequential(
      (0): Linear(in_features=240, out_features=72, bias=True)
      (1): ReLU()
      (2): Linear(in_features=

epoch 1: 100%|██████████| 282/282 [00:09<00:00, 31.12it/s]


validation loss: 0.0043107227500942015 at epoch 2
learning_rate: 0.002
model saved in ani_acrolein.pt


epoch 2: 100%|██████████| 282/282 [00:07<00:00, 36.76it/s]


validation loss: 0.004105910354190403 at epoch 3
learning_rate: 0.002
model saved in ani_acrolein.pt


...

validation loss: 0.001386098861694336 at epoch 999
learning_rate: 7.036874417766407e-05


epoch 999: 100%|██████████| 282/282 [00:08<00:00, 33.69it/s]


validation loss: 0.0014190933696097798 at epoch 1000
learning_rate: 7.036874417766407e-05


epoch 1000: 100%|██████████| 282/282 [00:07<00:00, 36.57it/s]

model loaded from ani_acrolein.pt


#Testing

In [ ]:
atoms = np.tile(np.array(1 * [6] + 1 * [8] + 2 * [6] + 4 * [1]), (5000, 1))
acrolein_test = molecular_database.from_numpy(coordinates_trans_test, atoms)

In [ ]:
acrolein_test.add_scalar_properties(epots_test, "energy")

In [ ]:
acrolein_test.add_xyz_derivative_properties(grads_test, "energy", "energy_grad")

In [ ]:
acrolein_test.size

5000

In [ ]:
mcd_test_mol = acrolein_test.copy()
model_ani.predict(
    molecular_database=mcd_test_mol,
    property_to_predict="energy",
    xyz_derivative_property_to_predict="energy_grad",
    batch_size=100,
)
# mcd_test_mol.get_properties("energy")
# mcd_test_mol.get_xyz_derivative_properties("energy_grad")
np.savez('test.npz', energy=np.array(mcd_test_mol.get_properties("energy")) /kcalpermol2eV, grad=np.array(mcd_test_mol.get_xyz_derivative_properties("energy_grad")) / kcalpermol2eV)

In [ ]:
mse_epot = np.mean(
    ((mcd_test_mol.get_properties("energy") - acrolein_test.get_properties("energy")))** 2)
print(np.sqrt(mse_epot))
mse_grad = np.mean(
    (
        (
            mcd_test_mol.get_xyz_derivative_properties("energy_grad")
            - acrolein_test.get_xyz_derivative_properties("energy_grad")
        )
    )
    ** 2
)
print(np.sqrt(mse_grad))

0.01805434643685519
0.13350999236934297


In [ ]:
mcd_train_mol = acrolein_train.copy()
model_ani.predict(
    molecular_database=mcd_train_mol,
    property_to_predict="energy",
    xyz_derivative_property_to_predict="energy_grad",
    batch_size=100,
)
# mcd_test_mol.get_properties("energy")
# mcd_test_mol.get_xyz_derivative_properties("energy_grad")
np.savez('train.npz', energy=np.array(mcd_train_mol.get_properties("energy")) /kcalpermol2eV, grad=np.array(mcd_train_mol.get_xyz_derivative_properties("energy_grad")) / kcalpermol2eV)

#Cis-Trans

In [ ]:
atoms = np.tile(np.array(1 * [6] + 1 * [8] + 2 * [6] + 4 * [1]), (20000, 1))
acrolein_cis_trans = molecular_database.from_numpy(coordinates_cis_trans, atoms)
acrolein_cis_trans.add_scalar_properties(energy_cis_trans.flatten() * kcalpermol2eV, "energy")
acrolein_cis_trans.add_xyz_derivative_properties(grads_cis_trans * kcalpermol2eV, "energy", "energy_grad")

mcd_cis_trans = acrolein_cis_trans.copy()

model_ani.predict(
    molecular_database=mcd_cis_trans,
    property_to_predict="energy",
    xyz_derivative_property_to_predict="energy_grad",
    batch_size=100,
)
# mcd_test_mol.get_properties("energy")
# mcd_test_mol.get_xyz_derivative_properties("energy_grad")
np.savez('cis_trans.npz', energy=np.array(mcd_cis_trans.get_properties("energy")) /kcalpermol2eV, grad=np.array(mcd_cis_trans.get_xyz_derivative_properties("energy_grad")) / kcalpermol2eV)

#Cis

In [ ]:
atoms = np.tile(np.array(1 * [6] + 1 * [8] + 2 * [6] + 4 * [1]), (50000, 1))
acrolein_cis = molecular_database.from_numpy(coordinates_cis, atoms)
acrolein_cis.add_scalar_properties(energy_cis.flatten() * kcalpermol2eV, "energy")
acrolein_cis.add_xyz_derivative_properties(grads_cis * kcalpermol2eV, "energy", "energy_grad")

mcd_cis = acrolein_cis.copy()

model_ani.predict(
    molecular_database=mcd_cis,
    property_to_predict="energy",
    xyz_derivative_property_to_predict="energy_grad",
    batch_size=100,
)
# mcd_test_mol.get_properties("energy")
# mcd_test_mol.get_xyz_derivative_properties("energy_grad")
np.savez('cis.npz', energy=np.array(mcd_cis.get_properties("energy")) /kcalpermol2eV, grad=np.array(mcd_cis.get_xyz_derivative_properties("energy_grad")) / kcalpermol2eV)